**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Oct 15 08:01:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 C++					      'page (1).gdoc'
'Catsss senka2.psd'			      'page (2).gdoc'
'Colab Notebooks'			      'page (3).gdoc'
'Dopis_2019-za opstine.docx'		       page.gdoc
'Dopis_2019-za opstine.gdoc'		      'Periša fekalije'
'Dušan Blanuša pasoš.pdf'		      'Pitanja iz junskog roka.docx'
'Dusan Blanusa predlog diplomskog Soft.docx'  'Pitanja iz junskog roka.gdoc'
 Fakultet				       PJISP
 Instalacije				       Poslovna
'ISA frontend'				      'Programski prevodioci'
 login.gdoc				      'PSW KT2 Demo'
'Members2019_fr (1).gsheet'		      'Salipurevic projekat.zip'
 Members2019_fr.gsheet			       SIMS
'Memristor 2020'			       Slike
'Metode teorija prvi kolokvijum'	       SPPuRV
 Mreze-test.gdoc			       sppurv1
'My Drive'				      'Sreten petrovaradin alfa'
'NLP sajtovi.gdoc'			       Startup
'Oet Marijin'				       Test-iz-teorije.gdoc
 OISISI					      'Video '
'OISISI od asistenta'			      'WEB programiranje'
 OP					       yolov3
'Operativni sistemi'			      

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14321, done.
remote: Total 14321 (delta 0), reused 0 (delta 0), pack-reused 14321
Receiving objects: 100% (14321/14321), 12.87 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (9772/9772), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "red_glass" > data/obj.names
!echo "green_glass" > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-27 16:27:07--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  1.41MB/s    in 1m 57s  

2020-09-27 16:29:05 (1.32 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/rgb (10).txt   
  inflating: data/obj/rgb (11).png   
  inflating: data/obj/rgb (11).txt   
  inflating: data/obj/rgb (12).png   
  inflating: data/obj/rgb (12).txt   
  inflating: data/obj/rgb (13).png   
  inflating: data/obj/rgb (13).txt   
  inflating: data/obj/rgb (15).png   
  inflating: data/obj/rgb (15).txt   
  inflating: data/obj/rgb (16).png   
  inflating: data/obj/rgb (16).txt   
  inflating: data/obj/rgb (17).png   
  inflating: data/obj/rgb (17).txt   
  inflating: data/obj/rgb (18).png   
  inflating: data/obj/rgb (18).txt   
  inflating: data/obj/rgb (19).png   
  inflating: data/obj/rgb (19).txt   
  inflating: data/obj/rgb (20).png   
  inflating: data/obj/rgb (20).txt   
  inflating: data/obj/rgb (22).png   
  inflating: data/obj/rgb (22).txt   
  inflating: data/obj/rgb (23).png   
  inflating: data/obj/rgb (23).txt   
  inflating: data/obj/rgb (24).png   
  inflating: data/obj/rgb (24).txt   
  inflating: 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:
                # Definisanje klase i koordinata objekata za svaki objekat iz txt fajla
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Upis u fajl
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1 0.641016 0.456250 0.113281 0.306944

['1', '0.641016', '0.456250', '0.113281', '0.306944']
0 data/obj/rgb (34).txt
1 0.641016 0.456250 0.113281 0.306944
1 0.468359 0.327083 0.117969 0.270833

['1', '0.468359', '0.327083', '0.117969', '0.270833']
0 data/obj/rgb (34).txt
1 0.468359 0.327083 0.117969 0.270833
1 0.912109 0.637500 0.164844 0.350000

['1', '0.912109', '0.637500', '0.164844', '0.350000']
0 data/obj/rgb (34).txt
1 0.912109 0.637500 0.164844 0.350000
1 0.050781 0.361111 0.100000 0.216667

['1', '0.050781', '0.361111', '0.100000', '0.216667']
0 data/obj/rgb (34).txt
1 0.050781 0.361111 0.100000 0.216667
0 0.807031 0.302778 0.118750 0.319444

['0', '0.807031', '0.302778', '0.118750', '0.319444']
0 data/obj/rgb (34).txt
0 0.807031 0.302778 0.118750 0.319444
0 0.287891 0.321528 0.130469 0.248611

['0', '0.287891', '0.321528', '0.130469', '0.248611']
0 data/obj/rgb (34).txt
0 0.287891 0.321528 0.130469 0.248611
0 0.379297 0.836806 0.164844 0.281944

['0', '0.379297', '0.836806', '

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")

In [ ]:
#Pravljenje training.txt fajla
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

FileNotFoundError: ignored

**6) Start the training**

In [ ]:
# Obucavanje mreze
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.722427, GIOU: 0.722429), Class: 0.974828, Obj: 0.204811, No Obj: 0.000069, .5R: 1.000000, .75R: 0.000000, count: 1, class_loss = 0.638294, iou_loss = 0.329153, total_loss = 0.967447 
 total_bbox = 1922614, rewritten_bbox = 0.004057 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000006, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.786789, GIOU: 0.770937), Class: 0.996817, Obj: 0.724892, No Obj: 0.007058, .5R: 1.000000, .75R: 0.571429, count: 7, class_loss = 1.274650, iou_loss = 2.335376, total_loss = 3.610026 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.378788, GIOU: 0.186230), Class: 0.981159, Obj: 0.119252, No Ob